In [1]:
import pandas as pd 

In [2]:
# Load cleaned data
data3=pd.read_csv('cleaned_data2_iotid23.csv')

In [3]:
data3.shape

(1230402, 35)

In [4]:
for column in data3.columns:
    print(f"Value counts for column: {column}")
    print(data3[column].value_counts())
    print("\n" + "=" * 50 + "\n")  # Separator for better readability


Value counts for column: Protocol
Protocol
6     815865
0     236808
17    177729
Name: count, dtype: int64


Value counts for column: Flow Duration
Flow Duration
2           8234
3           7602
1           6407
4           2212
9           1689
            ... 
18583617       1
10510736       1
21998999       1
9112427        1
146262         1
Name: count, Length: 447786, dtype: int64


Value counts for column: Total Fwd Packet
Total Fwd Packet
1       722482
2       318451
3        52570
4        42838
5         9989
         ...  
1799         1
3661         1
3626         1
3604         1
508          1
Name: count, Length: 1143, dtype: int64


Value counts for column: Total Length of Fwd Packet
Total Length of Fwd Packet
0.0        797832
2.0         69810
28.0        42304
6.0         24678
24.0        20122
            ...  
14944.0         1
75307.0         1
19257.0         1
40474.0         1
20340.0         1
Name: count, Length: 10480, dtype: int64


Value counts for col

## Model Building

### GNN+LSTM


In [6]:
%pip install torch torchvision torchaudio torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv networkx pandas numpy scikit-learn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -------------------- ------------------- 0.8/1.6 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.8/204.2 MB 3.7 MB/s eta 0:00:55
   ---------------------------------------- 1.6/204.2 MB 3.8 MB/s eta 0:00:54
   ---------------------------------------- 2.4/204.2 MB 3.7 MB/s eta

DEPRECATION: Loading egg at c:\users\emada\appdata\local\programs\python\python311\lib\site-packages\googletrans-3.0.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\torch_scatter
      copying torch_scatter\placeholder.py -> build\lib.win-amd64-cpython-311\torch_scatter
      copying torch_scatter\scatter.py -> build\lib.win-amd64-cpython-311\torch_scatter
      copying torch_scatter\segment_coo.py -> build\lib.win-amd64-cpython-311\torch_scatter
      copying torch_scatter\segment_csr.py -> build\lib.win-amd64-

In [7]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\emada\appdata\local\programs\python\python311\lib\site-packages\googletrans-3.0.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [6]:
df=pd.read_csv('cleaned_data2_iotid23.csv')

In [7]:
# Extract features and labels
features = df.iloc[:, :-1].values  # First 34 columns
labels = df.iloc[:, -1].values  # Label column

In [8]:
# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [10]:
# check for nan values
import numpy as np

print("Any NaN values:", np.isnan(features).any())
print("Any Infinite values:", np.isinf(features).any())
print("Max value in features:", np.max(features))
print("Min value in features:", np.min(features))

Any NaN values: False
Any Infinite values: True
Max value in features: inf
Min value in features: 0.0


In [18]:
features[np.isinf(features)] = np.nan  # Convert inf to NaN first
features = np.nan_to_num(features, nan=0.0)  # Replace NaNs with 0

In [19]:
# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)

In [20]:
# Split into train-test sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

In [30]:
from scipy.sparse import lil_matrix
from sklearn.metrics.pairwise import cosine_similarity


def create_adjacency_matrix(X, threshold=0.7, chunk_size=10000):
    n_samples = X.shape[0]
    adj_matrix = lil_matrix((n_samples, n_samples), dtype=np.int8)  # Sparse matrix

    for i in range(0, n_samples, chunk_size):
        for j in range(i, n_samples, chunk_size):  # Compute upper triangle
            sim_block = cosine_similarity(X[i : i + chunk_size], X[j : j + chunk_size])
            adj_block = (sim_block > threshold).astype(np.int8)
            adj_matrix[i : i + chunk_size, j : j + chunk_size] = adj_block
            adj_matrix[j : j + chunk_size, i : i + chunk_size] = adj_block  # Mirror

    return adj_matrix.tocsr()  # Convert to efficient sparse format


In [31]:
import networkx as nx


def construct_graph(X):
    adj_matrix = create_adjacency_matrix(X)
    G = nx.from_scipy_sparse_array(adj_matrix)  # Use sparse adjacency
    return G, adj_matrix


In [ ]:
# Apply graph transformation
X_train_graph = extract_graph_features(X_train, adj_matrix_train)
X_test_graph = extract_graph_features(X_test, create_adjacency_matrix(X_test))

# Reshape data for LSTM (samples, timesteps, features)
X_train_lstm = X_train_graph.reshape(X_train_graph.shape[0], 1, X_train_graph.shape[1])
X_test_lstm = X_test_graph.reshape(X_test_graph.shape[0], 1, X_test_graph.shape[1])

# Define LSTM model
model = Sequential(
    [
        LSTM(64, return_sequences=True, input_shape=(1, X_train_lstm.shape[2])),
        Dropout(0.3),
        LSTM(32, return_sequences=False),
        Dropout(0.3),
        Dense(16, activation="relu"),
        Dense(
            len(np.unique(y_train)), activation="softmax"
        ),  # Multi-class classification
    ]
)

# Compile model
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Train model
model.fit(
    X_train_lstm,
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test_lstm, y_test),
)

# Evaluate model
test_loss, test_acc = model.evaluate(X_test_lstm, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


In [ ]:
# Create graph for training data
G_train, adj_matrix_train = construct_graph(X_train)


# Convert adjacency matrix into a graph-based feature representation
def extract_graph_features(X, adj_matrix):
    """
    Transform features using graph neighborhood averaging.
    Each node's new feature is the mean of its neighbors' features.
    """
    new_features = np.zeros_like(X)
    for i in range(len(X)):
        neighbors = np.where(adj_matrix[i] == 1)[0]  # Get indices of connected nodes
        if len(neighbors) > 0:
            new_features[i] = np.mean(
                X[neighbors], axis=0
            )  # Mean of neighbors' features
        else:
            new_features[i] = X[i]  # If no neighbors, keep original features
    return new_features
